In [1]:
!python -V

Python 3.9.5


In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

2021-11-28 13:59:48.787055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-28 13:59:48.787091: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.7.0'

In [7]:
model = keras.models.load_model('dogs_cats_10_0.687.h5')

2021-11-28 14:02:03.752427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-28 14:02:03.752455: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-28 14:02:03.752481: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mark-laptop): /proc/driver/nvidia/version does not exist
2021-11-28 14:02:03.753177: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-28 14:02:03.819277: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 44859392 exceed

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [9]:
tflite_model = converter.convert()

2021-11-28 14:02:10.371644: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpmx_f0yfp/assets


2021-11-28 14:02:11.502595: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2021-11-28 14:02:11.502650: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2021-11-28 14:02:11.504103: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpmx_f0yfp
2021-11-28 14:02:11.506539: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2021-11-28 14:02:11.506566: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmpmx_f0yfp
2021-11-28 14:02:11.515349: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2021-11-28 14:02:11.729922: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmpmx_f0yfp
2021-11-28 14:02:11.750413: I tensorflow/cc/saved_model/loader.cc:283] SavedModel load for tags { serve }; Status: success: OK. Took 246599

In [11]:
with open('dogs_cats-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [12]:
!ls -lh

total 129M
-rw-rw-r-- 1 mark mark  86M Nov 25 07:02  dogs_cats_10_0.687.h5
-rw-rw-r-- 1 mark mark  43M Nov 28 14:04  dogs_cats-model.tflite
-rw-rw-r-- 1 mark mark 6.0K Nov 28 14:03 'ML Zoomcamp Week 9 Homework.ipynb'


In [14]:
import tensorflow.lite as tflite

In [15]:
interpreter = tflite.Interpreter(model_path='dogs_cats-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [16]:
input_index

0

In [17]:
output_index

13

In [18]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [19]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [20]:
img = download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg')

In [22]:
img = prepare_image(img, (150,150))

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [37]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [38]:
x = np.array(img)
X = np.array([x])

In [41]:
X = train_datagen.flow(X)

In [42]:
X[0]

array([[[[0.7058824 , 0.7725491 , 0.7490196 ],
         [0.69411767, 0.7607844 , 0.7294118 ],
         [0.6313726 , 0.69803923, 0.6666667 ],
         ...,
         [0.76470596, 0.85098046, 0.83921576],
         [0.68235296, 0.74509805, 0.74509805],
         [0.6745098 , 0.7686275 , 0.77647066]],

        [[0.5254902 , 0.5921569 , 0.5686275 ],
         [0.7372549 , 0.80392164, 0.7803922 ],
         [0.60784316, 0.6745098 , 0.6431373 ],
         ...,
         [0.79215693, 0.86666673, 0.8588236 ],
         [0.6313726 , 0.7019608 , 0.6392157 ],
         [0.78823537, 0.87843144, 0.854902  ]],

        [[0.6862745 , 0.7490196 , 0.7372549 ],
         [0.61960787, 0.6862745 , 0.6627451 ],
         [0.64705884, 0.7137255 , 0.68235296],
         ...,
         [0.6       , 0.65882355, 0.64705884],
         [0.627451  , 0.69411767, 0.654902  ],
         [0.7058824 , 0.7960785 , 0.7725491 ]],

        ...,

        [[0.5803922 , 0.61960787, 0.58431375],
         [0.4901961 , 0.5254902 , 0.5058824 ]

In [45]:
X[0].shape

(1, 150, 150, 3)

In [46]:
model.predict(X)

array([[0.77049136]], dtype=float32)